# <center>Big Data &ndash; Exercises</center>
## <center>Fall 2023 &ndash; Week 9 &ndash; ETH Zurich</center>
## <center>Spark Dataframes and Spark SQL, Moodle exercise</center>

# Preparation for the moodle exercise in Spark

In this jupyter notebook we are going to make the preprocessing part of the dataset that is going to be used in the graded exercise of this week.
It will be the same language game dataset as in exercise08.

1. Change to exercise09 repository

2. Start docker <br>
```docker-compose up -d```

3. Getting the data:
Follow the procedure that is described below. The dataset can be found here: http://data.greatlanguagegame.com.s3.amazonaws.com/confusion-2014-03-02.tbz2. 

More specifically do the following:
- download the data      :<br> ```wget http://data.greatlanguagegame.com.s3.amazonaws.com/confusion-2014-03-02.tbz2```
- extract the data       :<br> ```tar -jxvf confusion-2014-03-02.tbz2```

4. Extract the part of the dataset that we will work with in this exercise: `head -n 3000000 confusion-2014-03-02.json > confusion-part.json`
## More Info about the data
You can find more information about the dataset (as well as the schema and examples) in this link: http://lars.yencken.org/datasets/languagegame/

## Instructions:

In every query we ask you for three quantities: the query itself, the result of the query as well as the productivity time. That means the development time of each query (time elapsed before you start writing the query, and the time at which the correct, final query is ready). Note that the time part of every question is optional and not graded. In order to make easier the time recording we created two functions that do it automatically. Run the cell below in order to import the functions into the current notebook. Then before each query we will have a ```start_exercise()``` cell that you have to run in order to start time recording. After you have finished your query and you are sure about the answer run the ```finish_exercise()``` one to get the time measurement. 

In [3]:
import time

def start_exercise():
    global last
    last = time.time()
    
def finish_exercise():
    global last
    print("This exercise took {0}s".format(int(time.time()-last)))

## <center>1. Spark Dataframes</center>

Write queries for the same questions as last week, but this time using Spark Dataframes operations (the data loading will take a minute)

### 1.0. Data preprocessing

In [4]:
import json
from pyspark.sql import SparkSession
from pyspark import SparkConf

spark = SparkSession.builder.master('local').getOrCreate()
sc = spark.sparkContext

path = "confusion-2014-03-02/confusion-part.json"
dataset = spark.read.json(path).cache()

In [5]:
#test it out
dataset.limit(3).show()

+--------------------+-------+----------+---------+--------------------+---------+
|             choices|country|      date|    guess|              sample|   target|
+--------------------+-------+----------+---------+--------------------+---------+
|[Maori, Mandarin,...|     AU|2013-08-19|Norwegian|48f9c924e0d98c959...|Norwegian|
|[Danish, Dinka, K...|     AU|2013-08-19|    Dinka|af5e8f27cef9e689a...|    Dinka|
|[German, Hungaria...|     AU|2013-08-19|  Turkish|509c36eb58dbce009...|   Samoan|
+--------------------+-------+----------+---------+--------------------+---------+



## Assignment 1
Find the number of games where the guessed language and target language is Maltese.

In [6]:
start_exercise()

In [9]:
#Your code here
dataset.filter("target = 'Maltese' and target = guess").count()


11258

In [10]:
finish_exercise()

This exercise took 109s


## Assignment 2
Return the number of distinct "target" languages.

In [11]:
start_exercise()

In [12]:
#Your code here
dataset.select("target").distinct().count()

78

In [13]:
finish_exercise()

This exercise took 31s


## Assignment 3
Return the sample IDs (i.e., the *sample* field) of the bottom three games where the guessed language is correct (equal to the target one) ordered by date (descending), then by language (ascending), then by country (descending).

In [14]:
start_exercise()

In [17]:
dataset.select("sample").filter("guess == target").orderBy(
    dataset["date"].desc(), dataset["target"].asc(), dataset["country"].desc()
).limit(3).collect()

[Row(sample='fdf23d0a7063ba2fcef4b18eb7d57ad8'),
 Row(sample='00b85faa8b878a14f8781be334deb137'),
 Row(sample='1dd8e1883037c6305b87afe382c4feba')]

In [18]:
finish_exercise()

This exercise took 458s


## Assignment 4
Aggregate all games by country and guess language, counting the number of guesses for each group and return the frequencies of the two most frequent country/language combinations.

In [19]:
start_exercise()

In [21]:
from pyspark.sql.functions import desc

dataset.groupBy(["country", "guess"]).count().orderBy(desc("count")).select("count").limit(2).collect()

[Row(count=20932), Row(count=20780)]

In [22]:
finish_exercise()

This exercise took 143s


## Assignment 5
Find the fraction (between 0 and 1) of games where (the answer was correct && the correct guess was the first choice amongst the array of possible answers)

Please write the fraction rounding to 4 decimals (eg. 0.3320)

In [ ]:
start_exercise()

In [23]:
dataset.filter("choices[0] = target and target = guess").count() / float(dataset.count())

0.25385233333333335

In [24]:
finish_exercise()

This exercise took 312s


## Assignment 6
Sort the languages by decreasing overall percentage of correct guesses and return the first four languages.

In [25]:
start_exercise()

In [26]:
from pyspark.sql.functions import desc

correct_c = dataset.filter("target = guess").groupBy("target").count().withColumnRenamed("count", "correct_guesses")
total_c = dataset.groupBy("target").count().withColumnRenamed("count", "total_guesses")
joined_df = correct_c.join(total_c, "target")
joined_df.select("*", 
    (joined_df["correct_guesses"] / joined_df["total_guesses"]).alias("percentage")).orderBy(desc("percentage")).limit(4).show()

+-------+---------------+-------------+------------------+
| target|correct_guesses|total_guesses|        percentage|
+-------+---------------+-------------+------------------+
| French|          65754|        68371|0.9617235377572363|
| German|          64468|        67989|0.9482122107988057|
|Italian|          61369|        66769|0.9191241444382873|
|Russian|          58495|        64425|0.9079549864183158|
+-------+---------------+-------------+------------------+



In [27]:
finish_exercise()

This exercise took 144s


## Assignment 7
Return the number of games played on the last day.

In [28]:
start_exercise()

In [29]:
last_day = dataset.agg({"date": "max"}).collect()[0]["max(date)"]
dataset.filter(dataset["date"] == last_day).count()

87672

In [30]:
finish_exercise()

This exercise took 288s


## <center>2. Spark SQL</center>

Write Spark SQL queries for the same questions as earlier.

### 2.0. Data preprocessing

In [31]:
!pip install sparksql-magic

     |████████████████████████████████| 198 kB 5.2 MB/s eta 0:00:01


In [32]:
%load_ext sparksql_magic

In [33]:
path = "confusion-2014-03-02/confusion-part.json"
dataset = spark.read.json(path).cache()
dataset.registerTempTable("dataset")

In [34]:
%%sparksql
-- test it out
SELECT *
FROM dataset
LIMIT 3

choices,country,date,guess,sample,target
"['Maori', 'Mandarin', 'Norwegian', 'Tongan']",AU,2013-08-19,Norwegian,48f9c924e0d98c959d8a6f1862b3ce9a,Norwegian
"['Danish', 'Dinka', 'Khmer', 'Lao']",AU,2013-08-19,Dinka,af5e8f27cef9e689a070b8814dcc02c3,Dinka
"['German', 'Hungarian', 'Samoan', 'Turkish']",AU,2013-08-19,Turkish,509c36eb58dbce009ccf93f375358d53,Samoan


## Assignment 1
Find the number of games where the guessed language and target language is Maltese.

In [35]:
start_exercise()

In [52]:
%%sparksql
SELECT COUNT(*)
FROM dataset
WHERE guess = 'Maltese' AND target = 'Maltese'

count(1)
11258


In [40]:
finish_exercise()

This exercise took 165s


## Assignment 2
Return the number of distinct "target" languages.

In [41]:
start_exercise()

In [44]:
%%sparksql
SELECT COUNT(DISTINCT target)
FROM dataset

count(DISTINCT target)
78


In [45]:
finish_exercise()

This exercise took 109s


## Assignment 3
Return the sample IDs (i.e., the *sample* field) of the bottom three games where the guessed language is correct (equal to the target one) ordered by date (descending), then by language (ascending), then by country (descending).

In [46]:
start_exercise()

In [49]:
%%sparksql

SELECT sample
FROM dataset
WHERE target = guess
ORDER BY date DESC, target ASC, country DESC
LIMIT 3


sample
fdf23d0a7063ba2fcef4b18eb7d57ad8
00b85faa8b878a14f8781be334deb137
1dd8e1883037c6305b87afe382c4feba


In [50]:
finish_exercise()

This exercise took 172s


## Assignment 4
Aggregate all games by country and guess language, counting the number of guesses for each group and return the frequencies of the two most frequent country/language combinations.

In [51]:
start_exercise()

In [54]:
%%sparksql

SELECT country, guess, COUNT(*)
FROM dataset
GROUP BY country, guess
ORDER BY COUNT(*) DESC

only showing top 20 row(s)


country,guess,count(1)
US,German,20932
US,French,20780
US,Russian,20645
US,Italian,20172
US,Cantonese,19830
US,Mandarin,19697
US,Korean,19439
US,Spanish,19288
US,Ukrainian,19280
US,Vietnamese,18738


In [55]:
finish_exercise()

This exercise took 274s


## Assignment 5
Find the fraction (between 0 and 1) of games where (the answer was correct && the correct guess was the first choice amongst the array of possible answers)

Please write the fraction rounding to 4 decimals (eg. 0.3320)

In [56]:
start_exercise()

In [58]:
%%sparksql

SELECT (SELECT COUNT(*)
FROM dataset
WHERE choices[0] = target AND guess == target) / (SELECT COUNT(*)
FROM dataset)


(CAST(scalarsubquery() AS DOUBLE) / CAST(scalarsubquery() AS DOUBLE))
0.25385233333333335


In [59]:
finish_exercise()

This exercise took 278s


## Assignment 6
Sort the languages by decreasing overall percentage of correct guesses and return the first four languages.

In [60]:
start_exercise()

In [62]:
%%sparksql
SELECT target 
FROM (
    SELECT target, count(*) as correct
    FROM dataset
    WHERE target = guess
    GROUP BY target
) JOIN 
(
    SELECT target, count(*) as total
    FROM dataset
    GROUP BY target
) USING(target)
ORDER BY correct/total DESC
LIMIT 4

target
French
German
Italian
Russian


In [63]:
finish_exercise()

This exercise took 963s


## Assignment 7
Return the number of games played on the last day.

In [64]:
start_exercise()

In [65]:
%%sparksql

SELECT COUNT(*)
FROM dataset
WHERE date IN (SELECT date FROM dataset ORDER BY date desc LIMIT 1)

count(1)
87672


In [66]:
finish_exercise()

This exercise took 87s
